In [19]:
import numpy as np
import torch
from torch.utils.data import DataLoader , Dataset 

import matplotlib.pyplot as plt
from cfg import *
torch.manual_seed(6666)



use constantpad2d() to resize 

In [20]:
import cv2
import torchvision.transforms as T 
count = 0 

class Total_dataset(Dataset):
    def __init__(self, inputs: list, targets: list, transform=None) -> None:
        super().__init__()
        self.inputs = inputs
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self , index : int  , transform = None):
        #是否有辦法用path + file name取用data
        input_img = self.inputs[index]
        target_img = self.targets[index]
        
        x_data , y_data = cv2.imread(input_img) , cv2.imread(target_img)
        
        """global count 
        cv2.imwrite(f'./img_data/{count}.jpg' , x_data)
        print(type(x_data))
        print(type(y_data))
        count += 1"""
        
        #resize
        if x_data.shape != [512 , 512 , 3] :
            x_data = cv2.resize(x_data , (512 , 512))
        if y_data.shape != [512 , 512 , 3] :
            y_data = cv2.resize(y_data , (512 ,512))

        # turn to torch (typecasting)

        #data augmentaion if true
        if self.transform != None : 
            #x_data , y_data = self.transform(x_data , y_data)
            for t in self.transform :
                if t == T.Normalize([1 , 1 , 1] , [1 , 1 , 1]) :
                    """
                    #calculated in tensor type
                    x y mean0 tensor(0.0806) tensor(0.0015)
                    x y mean1 tensor(0.1362) tensor(0.0015)
                    x y mean2 tensor(0.1254) tensor(0.0015)
                    x y std0 tensor(0.1267) tensor(0.0270)
                    x y std01 tensor(0.1948) tensor(0.0270)
                    x y std2 tensor(0.1966) tensor(0.0270)
                    """
                    #normalized with whole dataset mean and std (not batchwise)
                    x_data = T.Normalize([0.0806 , 0.1362 , 0.1254] , [0.1267 , 0.1948 , 0.1966])
                    y_data = T.Normalize([0.0015 , 0.0015 , 0.0015] , [0.0270 , 0.0270 , 0.0270]) 
                else :
                    x_data = t(x_data)
                    y_data =t(y_data)
                
        #x_data, y_data = torch.from_numpy(x_data).type(torch.float32), torch.from_numpy(y_data).type(torch.float32)
        
        return x_data , y_data  #torch

        #torch or numpy to chose

        
        
    
        


In [21]:

#  currently using only nonPR (T1c)
import os
#non_T1C_data = "./data/PSPF_voc_data/non_PR/T1c/JPEGImages/" 
#non_T1C_target = "./data/PSPF_voc_data/non_PR/T1c/SegmentationClassPNG/" 

non_name_set = set()
PR_name_set = set()
name_set = set()
total_data_path = [non_T1C_data , non_T1_data , non_Flair_data , non_T2_data , PR_T1C_data , PR_T1_data , PR_Flair_data  , PR_T2_data]
count1 = 0
for folder_name in total_data_path :
    for name in os.listdir(folder_name) :
        name = name.strip('.jpg').strip('.json')
        if name == 'ipynb_checkpoint' :
            continue
        name_set.add(name)
        if count1 <= 3 :
            non_name_set.add(name)
        else :
            PR_name_set.add(name)
    count1 += 1
#print(name_set)


non_name_set2 = set()
PR_name_set2 = set()
name_set2 = set()
count2 = 0
total_target_path = [ non_T1C_target , non_T1_target , non_Flair_target  , non_T2_target , PR_T1C_target , PR_T1_target ,  PR_Flair_target , PR_T2_target ]
for folder_name2 in total_target_path :
    for name in os.listdir(folder_name2):
        name = name.strip('.png')
        if name == 'ipynb_checkpoint':
            continue
        name_set2.add(name)
        if count2 <= 3 :
            non_name_set2.add(name)
        else :
            PR_name_set2.add(name)
    count2 += 1
print(name_set2)


#same patients set (make sure no mismatch)
if name_set2 == name_set :
    print('success')
"""print(len(name_set))
print(len(name_set2))
print(len(non_name_set))
print(len(PR_name_set))"""
if non_name_set2 == non_name_set :
    print("fuck2021")
if PR_name_set2 == PR_name_set :
    print("fuck2023")
    


{'17244854', '28267864', '15525495', '27103788', '20483981', '30732894', '18765851', '80190941', '24237245', '33056266', '28358688', '11009708', '27237207', '27710844', '28248060', '29177415', '25825974', '15176128', '29301889', '26082459', '29729120', '29816081', '22296689', '26658418', '26073373', '31445157', '18456622', '20947834', '16113441', '31406726', '27607264', '13661917', '10027124', '30517836', '26434694', '29285707', '12261283', '26608196', '19425238', '15552568', '21105320', '52847319', '27931321', '30268702', '21372392', '14041674', '19315335', '27472711', '21381973', '13488672', '31001430', '11775010', '31046225', '30920239', '25385854', '30871154'}
success
fuck2021
fuck2023


In [22]:
#same patient (same index)
total_data_list = []
total_target_list =[]  
for name in non_name_set :
    name1 = name + ".jpg"
    name2 = name + ".png"
    total_data_list.append(os.path.join(non_T1C_data , name1))
    total_data_list.append(os.path.join(non_T1_data , name1))
    total_data_list.append(os.path.join(non_Flair_data , name1))
    total_data_list.append(os.path.join(non_T2_data , name1))
    total_target_list.append(os.path.join(non_T1C_target , name2))
    total_target_list.append(os.path.join(non_T1_target , name2))
    total_target_list.append(os.path.join(non_Flair_target , name2))
    total_target_list.append(os.path.join(non_T2_target , name2)) 

for name in PR_name_set :
    name1 = name + ".jpg"
    name2 = name + ".png"
    total_data_list.append(os.path.join(PR_T1C_data , name1))
    total_data_list.append(os.path.join(PR_T1_data , name1))
    total_data_list.append(os.path.join(PR_Flair_data , name1))
    total_data_list.append(os.path.join(PR_T2_data , name1))
    total_target_list.append(os.path.join(PR_T1C_target , name2))
    total_target_list.append(os.path.join(PR_T1_target , name2))
    total_target_list.append(os.path.join(PR_Flair_target , name2))
    total_target_list.append(os.path.join(PR_T2_target , name2))   

train_data_list = total_data_list[0 : int(len(total_data_list)*0.8)] #80%data
train_target_list =  total_target_list[0 : int(len(total_target_list)*0.8) ] #80%target

test_data_list = total_data_list[int(len(total_data_list)*0.8) : len(total_data_list)] #20%data
test_target_list = total_target_list[int(len(total_target_list)*0.8) : len(total_target_list)] #20%target

"""#img visualize
count , count2 = 0 , 0
for img1 in total_data_list:
    img1 = cv2.imread(img1)
    print(type(img1))
    plt.imshow(img1)
    plt.show()
    print(img1.shape)
    count += 1
    if count > 10 : break
for img2 in total_target_list:
    img2 = cv2.imread(img2)
    print(type(img2))
    plt.imshow(img2)
    plt.show()
    print(img2.shape)
    count2 += 1
    if count2 > 10: break
"""

#std and mean calculated
#total dataset
total_dataset = Total_dataset(total_data_list ,
                              total_target_list,
                              transform= [T.ToTensor()]
)
total_dataloader = DataLoader(total_dataset , batch_size= len(total_dataset) , shuffle= False)
x , y = next(iter(total_dataloader))
print("x y mean0" ,x[0].mean() , y[0].mean()) #ch1
print("x y mean1" ,x[1].mean() , y[1].mean()) #ch2
print("x y mean2" ,x[2].mean() , y[2].mean()) #ch3
print("x y std0" , x[0].std() , y[0].std()) #ch1
print("x y std01" , x[1].std() , y[1].std()) #ch2
print("x y std2" , x[2].std() , y[2].std() ) #ch3 

x y mean0 tensor(0.0925) tensor(0.0018)
x y mean1 tensor(0.1173) tensor(0.0018)
x y mean2 tensor(0.1497) tensor(0.0018)
x y std0 tensor(0.1365) tensor(0.0297)
x y std01 tensor(0.1797) tensor(0.0297)
x y std2 tensor(0.2085) tensor(0.0297)


data augmentation

In [28]:

#目前尚未 rescale 跟 normalize
train_dataset = Total_dataset(train_data_list,
                             train_target_list ,
                             transform= [T.ToTensor() , T.Normalize([1 , 1 , 1] , [1 , 1 , 1]) , 
                             #T.ColorJitter(brightness=.5, hue=.3) ,
                             T.RandomVerticalFlip(p=0.5) , 
                             T.RandomHorizontalFlip(p=0.5) , 
                             T.RandomAutocontrast(p = 0.4) ,
                             T.RandomRotation(degrees=(0, 360))
                             ])
                             
#transform should add                
training_dataloader = DataLoader(train_dataset,
                                 batch_size=1,
                                 shuffle=True)

test_dataset = Total_dataset(test_data_list ,
                            test_target_list ,
                            transform= [ T.ToTensor() , T.Normalize([1 , 1 , 1] , [1 , 1 , 1]) , ])

testing_dataloader = DataLoader(test_dataset,
                                 batch_size=1,
                                 shuffle=True)


while(True):

    train_x, train_y = next(iter(training_dataloader))
    if train_x == None and train_y == None :
        break
    print(f'train_x = shape: {train_x.shape}; type: {train_x.dtype}')
    print(f'train_y = shape: {train_y.shape}; type: {train_y.dtype}')
test_x , test_y = next(iter(testing_dataloader))

"""
#all img and label size is (512 , 512 , 3)
print(f'train_x = shape: {train_x.shape}; type: {train_x.dtype}')
print(train_x[0])
print(f'train_y = shape: {train_y.shape}; type: {train_y.dtype}')
print(f'train_x = min: {train_x.min()}; max: {train_x.max()}')
print(f'train_y = shape: {train_y.shape}; class: {train_y.unique()}; type: {train_y.dtype}')
"""


train_x = shape: torch.Size([1, 3, 512, 512]); type: torch.float32
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
train_y = shape: torch.Size([1, 3, 512, 512]); type: torch.float32
train_x = min: 0.0; max: 1.0
train_y = shape: torch.Size([1, 3, 512, 512]); clas

In [29]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras

#layer visualized
import tensorflow as tf
"""input_shape = (1 ,512, 512, 3)
x = tf.random.normal(input_shape)
"""


def unet(pretrained_weights = None,input_size = (512 , 512 , 3)):
    #Dropout(rate = 0.5) -> rate: Float between 0 and 1. Fraction of the input units to drop.
    #->  as to prevent from overfitting
    
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #pool1 -> (_ , 256 , 256 , 64)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #pool2 -> (_ , 128 , 128 , 128)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #pool3 -> (_ , 64 , 64 , 256)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    #pool4 -> (_ , 32 , 32 , 512)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
    #drop5 -> (_ ,  32, 32, 1024)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    #up6 (1, 64, 64, 512)
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    #conv6 ->(1, 64, 64, 512)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    #up7 (1, 128, 128, 256)
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    #conv7 (1, 128, 128, 256)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    #up8 (1, 256, 256, 128)
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    #up9 (1, 512, 512, 64)
    merge9 = concatenate([conv1,up9], axis = 3)
    #merge9 (1, 512, 512, 128)
    conv9 = Conv2D(64, 3, activation = 'relu', padding =  'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [30]:
#hyperparameters
batch_size = 10

In [31]:
"""training and saving"""
model = unet()
model_checkpoint = ModelCheckpoint('CMtumor_unet1.hdf5', monitor='loss',verbose=1, save_best_only=True)
print("model is fitting.......")
model.fit_generator(training_dataloader,steps_per_epoch=300,epochs=1,callbacks=[model_checkpoint])



model is fitting.......


C:\Users\user\anaconda3\envs\2dunet\lib\site-packages\ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


ValueError: Failed to find data adapter that can handle input: <class 'torch.utils.data.dataloader.DataLoader'>, <class 'NoneType'>